In [ ]:
# Project: AgentsVille Trip Planner
#
# Welcome to your final project! In this notebook, you'll build the "AgentsVille Trip Planner,"
# an AI-powered assistant that helps users plan trips to the imaginary city of AgentsVille.
# You will apply the prompting techniques and agentic reasoning concepts you've learned
# throughout the course.

# Project Goal:
# 1. Generate an initial, detailed travel itinerary based on user preferences.
# 2. Enhance this itinerary using an AI agent that can use (simulated) tools.

# --- 0. Setup ---
# Import necessary libraries and set up the OpenAI client and helper functions.

# Import necessary libraries
import os
import json
import re
from enum import Enum
from typing import List, Dict, Any, Optional, Literal

from openai import OpenAI
# Ensure you have pydantic installed: pip install pydantic
from pydantic import BaseModel, Field, ValidationError
from IPython.display import Markdown, display

# Attempt to import from project_lib, provide stubs if not found
try:
    from project_lib import (
        TravelPreferences, TravelItinerary, Activity, DayPlan, PackingListItem, # Pydantic Models
        available_tools, # Dictionary of tool schemas and functions
        # Tool implementations are called via available_tools dictionary
        LLMToolCall # Pydantic model for tool calls (if you choose to use it for internal validation)
    )
    print("Successfully imported from project_lib.py")
except ImportError:
    print("Warning: project_lib.py not found or some components are missing. Using placeholder Pydantic models and functions.\n",
          "Please ensure project_lib.py is in the same directory as this notebook for full functionality.")

    # Define dummy Pydantic models if project_lib is not available
    class TravelPreferences(BaseModel):
        destination: str
        duration_days: int
        travel_style: List[str]
        interests: List[str]
        budget: Literal["budget", "mid-range", "luxury"]
        specific_requests: Optional[str] = None

    class Activity(BaseModel):
        time: str
        description: str
        estimated_cost: Optional[str] = None
        details: Optional[str] = None

    class DayPlan(BaseModel):
        day_number: int
        theme: Optional[str] = None
        activities: List[Activity]

    class PackingListItem(BaseModel):
        item: str
        quantity: Optional[int] = 1
        notes: Optional[str] = None

    class TravelItinerary(BaseModel):
        trip_name: str
        destination: str
        duration_days: int
        daily_plans: List[DayPlan]
        suggested_packing_list: Optional[List[PackingListItem]] = None
        overall_estimated_budget: Optional[str] = None
        notes: Optional[str] = None

    # This LLMToolCall is for validating the *structure* of what the LLM intends to call.
    # The actual OpenAI response will have a slightly different structure for `tool_calls`.
    class LLMToolCall(BaseModel):
        name: str
        arguments: Dict[str, Any]

    # Dummy tool implementations (these would be in project_lib.py)
    def get_weather_forecast(location: str, date: str) -> Dict[str, Any]:
        print(f"[Tool Stub] Called get_weather_forecast for {location} on {date}")
        return {"forecast": "Sunny", "temperature": "22C", "location": location, "date": date}

    def search_activities(location: str, interests: List[str], date: Optional[str] = None) -> List[Dict[str, Any]]:
        print(f"[Tool Stub] Called search_activities for {location} with interests {interests} on {date}")
        return [{"name": "Museum of Agentic Design", "description": "Explore the history of AI agents."}]

    def find_restaurants(location: str, cuisine_type: Optional[str] = None, price_range: Optional[str] = None) -> List[Dict[str, Any]]:
        print(f"[Tool Stub] Called find_restaurants for {location}, cuisine {cuisine_type}, price {price_range}")
        return [{"name": "The Prompt Cafe", "cuisine": cuisine_type if cuisine_type else "various"}]

    def book_hotel(location: str, check_in_date: str, check_out_date: str, preferences: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        print(f"[Tool Stub] Called book_hotel for {location} from {check_in_date} to {check_out_date}")
        return {"booking_confirmation": "HOTEL-STUB-CONFIRMED", "hotel_name": "The Grand Agent Hotel"}

    available_tools = {
        "get_weather_forecast": {
            "function": get_weather_forecast,
            "description": "Get the weather forecast for a specific location and date.",
            "parameters": { # This is a JSON schema object
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The city and state, e.g., San Francisco, CA"},
                    "date": {"type": "string", "description": "The date for the forecast, in YYYY-MM-DD format."}
                },
                "required": ["location", "date"]
            }
        },
        "search_activities": {
            "function": search_activities,
            "description": "Searches for activities based on location, interests, and optionally a date.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The location to search for activities."},
                    "interests": {"type": "array", "items": {"type": "string"}, "description": "A list of interests to guide the activity search."},
                    "date": {"type": "string", "description": "Optional. The specific date for activities, in YYYY-MM-DD format."}
                },
                "required": ["location", "interests"]
            }
        },
        "find_restaurants": {
            "function": find_restaurants,
            "description": "Finds restaurants based on location, cuisine type, and price range.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The location to search for restaurants."},
                    "cuisine_type": {"type": "string", "description": "Optional. The desired type of cuisine (e.g., Italian, Mexican)."},
                    "price_range": {"type": "string", "description": "Optional. The desired price range (e.g., $, $$, $$$)."}
                },
                "required": ["location"]
            }
        },
        "book_hotel": {
            "function": book_hotel,
            "description": "Books a hotel based on location, dates, and preferences.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The location for the hotel."},
                    "check_in_date": {"type": "string", "description": "The check-in date in YYYY-MM-DD format."},
                    "check_out_date": {"type": "string", "description": "The check-out date in YYYY-MM-DD format."},
                    "preferences": {
                        "type": "object",
                        "description": "Optional. Dictionary of preferences like room type, amenities.",
                        "properties": { # Properties of the 'preferences' object
                             "room_type": {"type": "string", "description": "e.g., King, Queen, Suite"},
                             "amenities": {"type": "array", "items": {"type": "string"}, "description": "e.g., ['pool', 'gym']"}
                        }, # No required fields for preferences, making it truly optional
                    }
                },
                "required": ["location", "check_in_date", "check_out_date"]
            }
        }
    }

# OpenAI Client Setup
# TODO: Configure your OpenAI API Key
# Replace "**********" with your actual Vocareum OpenAI API Key.
# Alternatively, if your classroom workspace sets the OPENAI_API_KEY environment variable,
# you can use `api_key=os.getenv("OPENAI_API_KEY")`.

try:
    client = OpenAI(
        base_url="https://openai.vocareum.com/v1",
        api_key="**********"  # TODO: Replace "**********" with your key
    )
    print("OpenAI client configured (key needs to be set by student).")
except Exception as e:
    print(f"Error configuring OpenAI client: {e}")

# Define helper functions
class OpenAIModels(str, Enum):
    GPT_41_MINI = "gpt-4.1-mini"

MODEL = OpenAIModels.GPT_41_MINI # Using a specific model for consistency

def get_completion(
    messages: List[Dict[str, Any]],
    model: str = MODEL,
    temperature: float = 0.2, # Lower temperature for more deterministic structured output
    response_format: Optional[Dict[str, str]] = None,
    tools: Optional[List[Dict[str, Any]]] = None, # For native tool calling
    tool_choice: Optional[str] = None # For native tool calling ("auto" or {"type": "function", "function": {"name": "my_function"}})
) -> Optional[Any]: # Returns the OpenAI message object or content string
    """
    Function to get a completion from the OpenAI API.
    Can handle direct content responses or responses indicating tool calls.
    """
    try:
        completion_params = {
            "model": model,
            "messages": messages,
            "temperature": temperature,
        }
        if response_format:
            completion_params["response_format"] = response_format
        if tools:
            completion_params["tools"] = tools
        if tool_choice:
            completion_params["tool_choice"] = tool_choice

        response = client.chat.completions.create(**completion_params)
        message = response.choices[0].message

        # If the LLM decides to call a tool, message.tool_calls will be populated.
        # Otherwise, message.content will have the text response.
        return message
    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        return None

def print_messages(messages: List[Dict[str, Any]]):
    """Helper function to print messages, including tool calls and tool results."""
    for msg_idx, message in enumerate(messages):
        role = message.get('role', 'unknown').capitalize()
        content = message.get('content')

        display(Markdown(f"--- Message {msg_idx} ---"))
        display(Markdown(f"**{role}:**"))

        if content and isinstance(content, str): # Standard text content
            display(Markdown(content))

        # Handling OpenAI v1.x tool_calls attribute on assistant message
        if message.get('role') == 'assistant' and message.get('tool_calls'):
            display(Markdown("  *Assistant wants to call tools:*"))
            for tc in message['tool_calls']:
                # tc is a ChatCompletionMessageToolCall object
                function_info = tc.function
                display(Markdown(f"  - **Tool Call ID:** `{tc.id}`"))
                display(Markdown(f"    - **Function to Call:** `{function_info.name}`"))
                display(Markdown(f"    - **Arguments (JSON string):** `{function_info.arguments}`"))

        # Handling 'tool' role messages (results of tool execution)
        if message.get('role') == 'tool':
            display(Markdown(f"  *Result for Tool Call ID `{message.get('tool_call_id')}` (Function: `{message.get('name')}`):*"))
            if content: # Content here is the result from the tool
                 display(Markdown(f"    `{str(content)}`"))
        # Removed the extra "---" print to avoid double lines with the new "Message idx" line.

# --- 1. Defining User Preferences (Input) ---
user_preferences_data = {
    "destination": "AgentsVille",
    "duration_days": 3,
    "travel_style": ["culture", "history", "local experiences"],
    "interests": ["museums", "historical landmarks", "local markets", "street food"],
    "budget": "mid-range",
    "specific_requests": "I'd like to visit at least one major museum and try authentic local cuisine. I prefer using public transport or walking."
}

try:
    user_prefs = TravelPreferences(**user_preferences_data)
    print("User Preferences (Validated):")
    print(user_prefs.model_dump_json(indent=2))
except ValidationError as e:
    print(f"Error validating user preferences: {e}")
    user_prefs = None

# --- 2. Initial Itinerary Generation (CoT & Structured Output) ---
# Reference: TravelItinerary Pydantic Model (defined in project_lib.py or stub above)

# TODO: Craft the system prompt for initial itinerary generation.
# This prompt must instruct the LLM to act as a detailed travel planner for AgentsVille.
# It needs to consider all aspects of the `user_prefs`.
# The output MUST be a JSON object that validates against the `TravelItinerary` Pydantic model.
# Remind it to ONLY output the JSON object.
# Apply principles of Chain-of-Thought by asking for a comprehensive, well-reasoned plan.

system_prompt_generate_itinerary = """
********** TODO: DEFINE YOUR SYSTEM PROMPT FOR ITINERARY GENERATION HERE **********
Key elements to include:
- Role: Expert Travel Planner for AgentsVille
- Task: Create a comprehensive, day-by-day itinerary.
- Input: User preferences (will be provided in user message).
- Output: A single JSON object strictly conforming to the TravelItinerary schema.
  (You might want to paste the Pydantic model's field names as a reminder of the structure here for the LLM,
   but the core instruction is to match the schema it will be validated against.)
- Guidance: Consider all preferences (duration, style, interests, budget, specific requests).
           Plan logically. Include activities, packing list, budget estimate, notes.
           Output ONLY the JSON. No other text.
"""

user_prompt_itinerary_request = f"""
Please generate a travel itinerary based on the following preferences:
{user_prefs.model_dump_json(indent=2) if user_prefs else "User preferences not available."}
"""

print("\\nGenerating initial itinerary...")
initial_itinerary: Optional[TravelItinerary] = None
raw_llm_itinerary_json_str = None

if user_prefs:
    messages_itinerary = [
        {"role": "system", "content": system_prompt_generate_itinerary},
        {"role": "user", "content": user_prompt_itinerary_request}
    ]
    # Requesting JSON object output directly from the API
    llm_message_for_itinerary = get_completion(messages_itinerary, response_format={"type": "json_object"})

    if llm_message_for_itinerary and isinstance(llm_message_for_itinerary.content, str):
        raw_llm_itinerary_json_str = llm_message_for_itinerary.content
        try:
            initial_itinerary_json = json.loads(raw_llm_itinerary_json_str)
            initial_itinerary = TravelItinerary(**initial_itinerary_json)
            print("\\nInitial Itinerary Generated and Validated Successfully!")
            display(Markdown(f"### Trip Name: {initial_itinerary.trip_name}"))
            # For full details, you can print:
            # print(initial_itinerary.model_dump_json(indent=2))
        except (ValidationError, json.JSONDecodeError) as e:
            print(f"\\nError processing itinerary JSON: {e}")
            print(f"\\nLLM Response was (string):\\n{raw_llm_itinerary_json_str}")
    else:
        print("\\nFailed to generate initial itinerary (LLM response was not direct content or was None).")
else:
    print("User preferences not loaded, cannot generate itinerary.")


# --- 3. Tool-Using Agent for Enhancements (ReAct) ---
# Create an AI agent that uses tools to answer follow-up questions or enhance the itinerary.

print("\\nSchema for 'search_activities' tool (from available_tools dictionary):")
if "search_activities" in available_tools:
    print(json.dumps(available_tools['search_activities']['parameters'], indent=2))

# TODO: Craft the system prompt for the ReAct agent.
# This is a challenging prompt. You need to instruct the LLM on:
# - Its role (AI Travel Assistant for AgentsVille).
# - The context (it has an existing itinerary, needs to answer questions or enhance it).
# - The THINK-ACT-OBSERVE cycle.
#   - THINK: Plan, reason, decide on tool.
#   - ACT: Output *only* a JSON list of tool calls if using tools (e.g., `[{"name": "tool_name", "arguments": {...}}]`),
#          OR output `FINAL ANSWER:` followed by text if answering directly.
# - The list of AVAILABLE TOOLS: Provide their names, descriptions, and parameter schemas
#   (you'll need to format this from the `available_tools` dictionary).
# - Clear EXAMPLES of both a tool-calling interaction and a direct final answer.

system_prompt_react_agent = f"""
********** TODO: DEFINE YOUR REACT AGENT SYSTEM PROMPT HERE **********
Key sections to include:
1. Role and Goal.
2. Detailed instructions on the THINK-ACT-OBSERVE cycle.
   - Emphasize the exact output format for ACT (tool calls as JSON list OR FINAL ANSWER: text).
3. The `AVAILABLE TOOLS` section:
   You should dynamically build this part of the prompt by iterating through the
   `available_tools` dictionary and formatting each tool's name, description,
   and parameters (JSON schema) clearly for the LLM.
4. Clear examples of interaction flows (one with a tool call, one with a direct final answer).
"""

# --- Simulate ReAct Agent Interaction ---
react_messages: List[Dict[str, Any]] = [
    {"role": "system", "content": system_prompt_react_agent}
]

if initial_itinerary:
    current_itinerary_str = initial_itinerary.model_dump_json(indent=2)
    follow_up_request = "Can you find a unique local craft workshop for the afternoon of Day 2? Also, what will the weather be like on Day 1 of my trip, assuming Day 1 is 2025-10-26?"

    user_message_content_for_react = f"""Here is the current travel itinerary:
```json
{current_itinerary_str}

My follow-up request is: {follow_up_request}
"""
    react_messages.append({"role": "user", "content": user_message_content_for_react})
else:
    print("Initial itinerary not available. Using a general request for ReAct agent.")
    react_messages.append({"role": "user", "content": "Help me plan a 3-day cultural trip to AgentsVille for a mid-range budget. What are some must-see historical landmarks?"})


MAX_TURNS = 7
final_answer_received = False

print("\\n--- Starting ReAct agent interaction ---")
if len(react_messages) > 1:
    print_messages([react_messages[-1]])
else:
    print("Warning: No initial user message for ReAct agent beyond system prompt.")


for turn in range(MAX_TURNS):
    if len(react_messages) <= 1 : # Should not happen if user message was added
        print("Stopping ReAct loop: No user query to process.")
        break
            
    print(f"\\n--- Agent Turn {turn + 1} ---")
    
    # Get LLM's response (this is an OpenAI Message object)
    llm_response_message = get_completion(react_messages)

    if not llm_response_message:
        print("Agent did not provide a response. Ending interaction.")
        break
    
    # The llm_response_message from OpenAI v1.x+ client contains `content` and `tool_calls`.
    # Add the assistant's entire message object to history.
    # The `tool_calls` attribute will be a list of ChatCompletionMessageToolCall objects.
    # We need to convert this to the dict format if we were to send it back to older API versions,
    # but for OpenAI v1.x, we can append the message object directly if the API expects it.
    # For clarity and explicit structure in `react_messages`, we'll construct a dict.
    
    assistant_response_for_history = {"role": "assistant"}
    if llm_response_message.content:
        assistant_response_for_history["content"] = llm_response_message.content
    if llm_response_message.tool_calls:
        assistant_response_for_history["tool_calls"] = [
            {"id": tc.id, "type": tc.type, "function": {"name": tc.function.name, "arguments": tc.function.arguments}}
            for tc in llm_response_message.tool_calls
        ]
    
    react_messages.append(assistant_response_for_history)
    print_messages([assistant_response_for_history]) # Display AI's full response (THINK/ACT part)

    # TODO: Implement the ReAct Loop Logic.
    # This is where you will apply the skills learned in the ReAct exercise.
    #
    # 1. Check for "FINAL ANSWER:" in `llm_response_message.content`.
    #    If found, set `final_answer_received = True` and `break`.
    #
    # 2. If `llm_response_message.tool_calls` is present:
    #    - This means the LLM wants to call one or more tools.
    #    - Iterate through each `tool_call` in `llm_response_message.tool_calls`.
    #        - Get the `tool_name` (from `tool_call.function.name`).
    #        - Get the `tool_id` (from `tool_call.id`).
    #        - Get the `tool_arguments_str` (from `tool_call.function.arguments`, which is a JSON string).
    #        - Parse `tool_arguments_str` into a Python dictionary (`json.loads()`).
    #        - Look up `tool_name` in your `available_tools` to get the actual Python tool function.
    #        - Execute the tool: `tool_result = tool_function(**parsed_arguments)`. Handle potential errors.
    #        - Convert `tool_result` back to a JSON string.
    #        - Construct a 'tool' role message to append to `react_messages`:
    #          `{"role": "tool", "tool_call_id": tool_id, "name": tool_name, "content": tool_result_json_string}`
    #          (The 'name' field in the tool role message is for matching the function name, as per OpenAI spec).
    #
    # 3. If it's not a final answer and not a tool call (i.e., `llm_response_message.content` is text but no "FINAL ANSWER:"):
    #    The LLM might be asking for clarification or made an error. For this project,
    #    you can append a generic "OBSERVATION: Your response was not a FINAL ANSWER or a tool call. Please proceed."
    #    message from the 'user' role to `react_messages` to prompt the LLM again.

    # --- START TODO: Implement ReAct Loop Logic (Parsing ACT, Calling Tools, Appending Observations) ---
    
    # Student needs to fill this logic based on the ReAct exercise.
    # Key parts:
    # - Check for "FINAL ANSWER:" in llm_response_message.content
    # - If not, check for llm_response_message.tool_calls
    #   - If tool_calls exist:
    #     - For each tool_call:
    #       - Get tool_name, tool_id, arguments_json_string
    #       - Parse arguments_json_string to a dict
    #       - Find the corresponding python function from available_tools
    #       - Execute the python function with the arguments
    #       - Create a 'tool' role message with tool_call_id, name, and stringified result
    #       - Append this 'tool' message to react_messages
    # - If neither FINAL ANSWER nor tool_calls, append a generic user observation to try and guide the LLM.

    # Placeholder for student implementation:
    if llm_response_message.content and "FINAL ANSWER:" in llm_response_message.content:
        final_answer_received = True
        print("\\nAgent provided FINAL ANSWER.")
        break
    elif llm_response_message.tool_calls:
        # This section is critical for the student to implement correctly
        # based on their understanding of the ReAct exercise and OpenAI tool calling.
        print("  (Student TODO: Implement tool call execution and observation appending here)")
        # As a simple stub for now, let's add a generic observation if tools were called but not handled
        # This part MUST be replaced by the student with actual tool execution logic.
        if not final_answer_received: # Only if we haven't already decided to break
             for tc_obj in llm_response_message.tool_calls:
                # This is a placeholder observation. Student needs to call the actual tool.
                stub_tool_observation = {
                    "role": "tool",
                    "tool_call_id": tc_obj.id,
                    "name": tc_obj.function.name,
                    "content": json.dumps({"status": "success", "message": f"Tool {tc_obj.function.name} called (student needs to implement actual execution)."})
                }
                react_messages.append(stub_tool_observation)
                print_messages([stub_tool_observation])

    else: # No FINAL ANSWER and no tool_calls
        print("  Agent did not call a tool or provide a final answer. Adding a generic observation.")
        generic_observation = {"role": "user", "content": "OBSERVATION: Your response did not include an 'ACT:' section with tool calls or a 'FINAL ANSWER:'. Please either call a tool or provide a final answer."}
        react_messages.append(generic_observation)
        print_messages([generic_observation])

    # --- END TODO: Implement ReAct Loop Logic ---

if not final_answer_received:
    print(f"\\nAgent did not provide a final answer after {MAX_TURNS} turns.")


# --- 4. Final Output and Reflection ---
print("\\n--- Full ReAct Agent Conversation History (if run) ---")
if initial_itinerary or len(react_messages) > 1 :
    print_messages(react_messages)

# TODO: If the agent modified the itinerary or provided specific information in its FINAL ANSWER,
# you might want to parse and display that here.

# --- Reflection Questions: ---
# (Keep the reflection questions as provided in the original starter)

# --- Congratulations! ---
# (Keep the congratulations message as provided)

  ```